In [ ]:
!pip install tensorflow

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
cmd_vel_data = pd.read_csv('cmd_vel_data.csv')
odom_data = pd.read_csv('odom_data_file.csv')
obstacle_data = pd.read_csv('obstacle_array_data.csv')
target_data = pd.read_csv('target_data.csv')

In [ ]:
lidar_input = obstacle_data[['sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector6', 'sector7', 'sector8', 'sector9', 'sector10', 'sector11', 'sector12', 'sector13', 'sector14', 'sector15', 'sector16']].values
odom_input = odom_data[['x_coor', 'y_coor', 'z_coor']].values
#odom_input = Input(shape=(3,), name='odom_input')
y_train_cmd_vel = cmd_vel_data[['linear_x', 'linear_y', 'angular_z']].values
target_input = target_data[['x_val', 'y_val']].values

In [ ]:
obstacle_data


,sector1,sector2,sector3,sector4,sector5,sector6,sector7,sector8,sector9,sector10,sector11,sector12,sector13,sector14,sector15,sector16
0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1614,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1615,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
cmd_vel_data1 = pd.read_csv('cmd_vel_data2.csv')
odom_data1 = pd.read_csv('odom_data_file2.csv')
obstacle_data1 = pd.read_csv('obstacle_array_data2.csv')
target_data1 = pd.read_csv('target_data2.csv')

In [ ]:
odom_data1

,x_coor,y_coor,z_coor
0,2.178185,-0.258020,0
1,2.178185,-0.258020,0
2,2.178185,-0.258020,0
3,2.178185,-0.258020,0
4,2.178185,-0.258020,0
...,...,...,...
2107,4.510715,2.446353,0
2108,4.526989,2.459800,0
2109,4.526989,2.459800,0
2110,4.526989,2.459800,0


In [ ]:

odom_input_test = odom_data1[['x_coor', 'y_coor', 'z_coor']].values
lidar_input_test = obstacle_data1[['sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector6', 'sector7', 'sector8', 'sector9', 'sector10', 'sector11', 'sector12', 'sector13', 'sector14', 'sector15', 'sector16']].values
y_train_cmd_vel_test = cmd_vel_data1[['linear_x', 'linear_y', 'angular_z']].values
target_input_test = target_data1[['x_val', 'y_val']].values
#concatenated_test_input = np.concatenate((lidar_input_test, odom_input_test,  target_input_test), axis=1)


In [ ]:
odom_input_test

array([[ 2.17818546, -0.25802046,  0.        ],
       [ 2.17818546, -0.25802046,  0.        ],
       [ 2.17818546, -0.25802046,  0.        ],
       ...,
       [ 4.52698946,  2.45979953,  0.        ],
       [ 4.52698946,  2.45979953,  0.        ],
       [ 4.52698946,  2.45979953,  0.        ]])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
lidar_train, lidar_test, \
odom_train, odom_test, \
target_train, target_test, \
cmd_vel_train, cmd_vel_test = train_test_split(lidar_input, odom_input, target_input, y_train_cmd_vel, test_size=0.2, random_state=42)

In [ ]:
def create_model():
    lidar_input_hash = layers.Input(shape=(lidar_input.shape[1],))
    odom_input_hash = layers.Input(shape=(odom_input.shape[1],))
    target_input_hash = layers.Input(shape=(target_input.shape[1],))

    # Example layers
    x = layers.Dense(64, activation='relu')(lidar_input_hash)
    x = layers.Dense(64, activation='relu')(odom_input_hash)
    x = layers.Dense(64, activation='relu')(target_input_hash)
    combined = layers.concatenate([x, lidar_input_hash, odom_input_hash, target_input_hash])

    output = layers.Dense(y_train_cmd_vel.shape[1], activation='softmax')(combined)

    model = models.Model(inputs=[lidar_input_hash, odom_input_hash, target_input_hash], outputs=output)
    return model

# Create the model
model = create_model()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit([lidar_train, odom_train, target_train], cmd_vel_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
33/33 [==============================] - 1s 15ms/step - loss: 0.2759 - accuracy: 0.2737 - val_loss: 0.0453 - val_accuracy: 0.9691
Epoch 2/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0447 - accuracy: 0.9826 - val_loss: 0.0465 - val_accuracy: 0.9691
Epoch 3/10
33/33 [==============================] - 0s 5ms/step - loss: 0.0418 - accuracy: 0.9826 - val_loss: 0.0446 - val_accuracy: 0.9691
Epoch 4/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0403 - accuracy: 0.9826 - val_loss: 0.0431 - val_accuracy: 0.9691
Epoch 5/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0395 - accuracy: 0.9826 - val_loss: 0.0423 - val_accuracy: 0.9691
Epoch 6/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0386 - accuracy: 0.9826 - val_loss: 0.0417 - val_accuracy: 0.9691
Epoch 7/10
33/33 [==============================] - 0s 3ms/step - loss: 0.0381 - accuracy: 0.9826 - val_loss: 0.0412 - val_accuracy: 0.9691
Epoch 8/10
33/33 [=

In [ ]:
loss, accuracy = model.evaluate([lidar_test, odom_test, target_test], cmd_vel_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
model.save("cmd_vel_prediction_model.h5")

11/11 [==============================] - 0s 2ms/step - loss: 0.0401 - accuracy: 0.9815
Test Loss: 0.04012415558099747
Test Accuracy: 0.9814814925193787


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
cmd_vel_data_ag = pd.read_csv('cmd_vel_data1.csv')
odom_data_ag = pd.read_csv('odom_data_file1.csv')
obstacle_data_ag = pd.read_csv('obstacle_array_data1.csv')
target_data_ag = pd.read_csv('target_data1.csv')

In [ ]:
lidar_input_ag = obstacle_data_ag[['sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector6', 'sector7', 'sector8', 'sector9', 'sector10', 'sector11', 'sector12', 'sector13', 'sector14', 'sector15', 'sector16']].values
odom_input_ag = odom_data_ag[['x_coor', 'y_coor', 'z_coor']].values
#odom_input = Input(shape=(3,), name='odom_input')
y_train_cmd_vel_ag = cmd_vel_data_ag[['linear_x', 'linear_y', 'angular_z']].values
target_input_ag = target_data_ag[['x_val', 'y_val']].values

In [ ]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit([lidar_input_ag, odom_input_ag, target_input_ag], y_train_cmd_vel_ag, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
33/33 [==============================] - 1s 11ms/step - loss: 0.0746 - accuracy: 0.9659 - val_loss: 0.0540 - val_accuracy: 1.0000
Epoch 2/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0702 - accuracy: 0.9971 - val_loss: 0.0571 - val_accuracy: 1.0000
Epoch 3/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0688 - accuracy: 0.9971 - val_loss: 0.0565 - val_accuracy: 1.0000
Epoch 4/10
33/33 [==============================] - 0s 3ms/step - loss: 0.0678 - accuracy: 0.9766 - val_loss: 0.0581 - val_accuracy: 1.0000
Epoch 5/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0671 - accuracy: 0.9581 - val_loss: 0.0592 - val_accuracy: 1.0000
Epoch 6/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0664 - accuracy: 0.9522 - val_loss: 0.0591 - val_accuracy: 1.0000
Epoch 7/10
33/33 [==============================] - 0s 4ms/step - loss: 0.0659 - accuracy: 0.9259 - val_loss: 0.0608 - val_accuracy: 1.0000
Epoch 8/10
33/33 [=

In [ ]:
loss, accuracy = model.evaluate([lidar_input_test, odom_input_test, target_input_test], y_train_cmd_vel_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

66/66 [==============================] - 0s 2ms/step - loss: 0.1163 - accuracy: 0.7154
Test Loss: 0.11628355085849762
Test Accuracy: 0.7154356241226196


In [ ]:
model.save("cmd_vel_prediction_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
lidar_input_ag


array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 0, 1]])